In [ ]:
#
# ==================== GLOVE =================
#

In [ ]:
from gensim.models import KeyedVectors

fname = "../../../LMT4NLP/data/glove/glove.6B.300d.txt"
glove = KeyedVectors.load_word2vec_format(fname, no_header=True)
glove.vectors.shape

In [ ]:
# king - man + woman
glove.most_similar(positive=["king", "woman"], negative=["man"])

In [ ]:
# berlin - germany + australia
glove.most_similar(positive=["berlin", "australia"], negative=["germany"])

In [ ]:
normed_vectors = glove.get_normed_vectors()
normed_vectors.shape

In [ ]:
def most_similar_words(word, vectors, index_to_key, key_to_index, topn=10):
    # retrieve word_id corresponding to given word
    word_id = key_to_index[word]
    # retrieve embedding for given word from the vectors matrix
    embedding = vectors[word_id]
    # calculate similarities to all words in the input vocabulary (vectors)
    similarities = vectors @ embedding
    # get word_ids in ascending order with respect to similarity score
    ids_ascending = similarities.argsort()
    # reverse word_ids
    ids_descending = ids_ascending[::-1]
    # get boolean array with element corresponding to word_id set to false
    mask = ids_descending != word_id
    # obtain new array of indices that does not contain word_id
    # (otherwise the most similar word to the argument would be the argument itself)
    ids_descending = ids_descending[mask]
    # get topn word_ids
    top_ids = ids_descending[:topn]
    # retrieve topn words with their corresponding similarity score
    top_words = [(index_to_key[i], similarities[i]) for i in top_ids]
    # return results
    return top_words

In [ ]:
vectors = glove.get_normed_vectors()
index_to_key = glove.index_to_key
key_to_index = glove.key_to_index
most_similar_words("cactus", vectors, index_to_key, key_to_index)

In [ ]:
import numpy as np
from numpy.linalg import norm

def analogy(positive, negative, vectors, index_to_key, key_to_index, topn=10):
    # find ids for positive and negative words
    pos_ids = [key_to_index[w] for w in positive]
    neg_ids = [key_to_index[w] for w in negative]
    given_word_ids = pos_ids + neg_ids
    # get embeddings for positive and negative words
    pos_emb = vectors[pos_ids].sum(axis=0)
    neg_emb = vectors[neg_ids].sum(axis=0)
    # get embedding for analogy
    emb = pos_emb - neg_emb
    # normalize embedding
    emb = emb / norm(emb)
    # calculate similarities to all words in out vocabulary
    similarities = vectors @ emb
    # get word_ids in ascending order with respect to similarity score
    ids_ascending = similarities.argsort()
    # reverse word_ids
    ids_descending = ids_ascending[::-1]
    # get boolean array with element corresponding to any of given_word_ids set to false
    given_words_mask = np.isin(ids_descending, given_word_ids, invert=True)
    # obtain new array of indices that doesn't contain any of the given_word_ids
    ids_descending = ids_descending[given_words_mask]
    # get topn word_ids
    top_ids = ids_descending[:topn]
    # retrieve topn words with their corresponding similarity score
    top_words = [(index_to_key[i], similarities[i]) for i in top_ids]
    # return results
    return top_words

In [ ]:
positive = ["king", "woman"]
negative = ["man"]
vectors = glove.get_normed_vectors()
index_to_key = glove.index_to_key
key_to_index = glove.key_to_index
analogy(positive, negative, vectors, index_to_key, key_to_index)